## Timeseries Modeling - Sarimax
### Objective:
The objective of the notebook is to -
* Backtest on all hyperparameters of SARIMAX provided in the config
* Find the best set of hyperparameters using the metric provided in the config

### Auto ARIMA
p,d,q values can be tuned in two different ways
- Using Auto ARIMA which helps us to infer the best p,d,q values from the data using minimum AIC value
- Using the "order" hyperparameter of SARIMAX constructor in the config file which loops through all possible orders and finds the best one using Backtesting. This is a time-consuming process

In [0]:
import yaml
import inspect
import glob
import numpy as np
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from distutils.command.config import config
from tqdm.auto import tqdm
from datetime import timedelta
from datetime import datetime
import mlflow
from sklearn.metrics import mean_absolute_error,mean_squared_error
import os
import logging
import pmdarima as pm
import dotsi

In [0]:
# logging part
p_dir = "/tmp/"
log_file = "Sarimax_hyperparameter_tuning" + " (" +datetime.today().strftime('%Y-%m-%d-%H-%M-%S')+ ").log"

# SARIMAX - Hyperparameter tuning logs
logger = logging.getLogger('custom_log')
logger.setLevel(logging.DEBUG)

# Applying necessary formatter
fh = logging.FileHandler(p_dir+log_file)
formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
fh.setFormatter(formatter)   
logger.addHandler(fh)

In [0]:
# Getting the default settings of hyperparameters. Used to check that user-provided hyperparameters must always be a subset of these.
def get_default_args(func) -> dict:
    """Function to get the default values of the hyperparameters for the given algorithm

    Parameters
    ----------
    func : constructor of the respective algorithm
        The name of the algorithm (Eg: Prophet,SARIMAX)

    Returns
    -------
    dict
        returns a dictionary of hyperparameters and the corresponding default values for the given algorithm
    """
    signature = inspect.signature(func)
    return {
        k: v.default if v.default is not inspect.Parameter.empty else None
        for k, v in signature.parameters.items()
        if k != 'self'
    }

#### Configurable Hyperparameters
The following are the possible hyperparameters that can be tuned for SARIMAX. The preferred hyperparameters, and their respective search spaces, over which tuning is to be done need to be mentioned in the config file.

In [0]:
# Default values for the hyperparameters in SARIMAX
default_hpps_fit = get_default_args(SARIMAX.fit)
default_hpps_init = get_default_args(SARIMAX.__init__)

default_hpps_arima = get_default_args(pm.auto_arima)

print("Hyperparameters under fit\n",default_hpps_fit,"\n\nHyperparameters under init\n",default_hpps_init,"\n\nHyperparameters for  Auto Arima\n",default_hpps_arima)

Hyperparameters under fit
 {'start_params': None, 'transformed': True, 'includes_fixed': False, 'cov_type': None, 'cov_kwds': None, 'method': 'lbfgs', 'maxiter': 50, 'full_output': 1, 'disp': 5, 'callback': None, 'return_params': False, 'optim_score': None, 'optim_complex_step': None, 'optim_hessian': None, 'flags': None, 'low_memory': False, 'kwargs': None} 

Hyperparameters under init
 {'endog': None, 'exog': None, 'order': (1, 0, 0), 'seasonal_order': (0, 0, 0, 0), 'trend': None, 'measurement_error': False, 'time_varying_regression': False, 'mle_regression': True, 'simple_differencing': False, 'enforce_stationarity': True, 'enforce_invertibility': True, 'hamilton_representation': False, 'concentrate_scale': False, 'trend_offset': 1, 'use_exact_diffuse': False, 'dates': None, 'freq': None, 'missing': 'none', 'validate_specification': True, 'kwargs': None} 

Hyperparameters for Auto Arima
 {'y': None, 'X': None, 'start_p': 2, 'd': None, 'start_q': 2, 'max_p': 5, 'max_d': 2, 'max_q': 5, 'start_P': 1, 'D': None, 'start_Q': 1, 'max_P': 2, 'max_D': 1, 'max_Q': 2, 'max_order': 5, 'm': 1, 'seasonal': True, 'stationary': False, 'information_criterion': 'aic', 'alpha': 0.05, 'test': 'kpss', 'seasonal_test': 'ocsb', 'stepwise': True, 'n_jobs': 1, 'start_params': None, 'trend': None, 'method': 'lbfgs', 'maxiter': 50, 'offset_test_args': None, 'seasonal_test_args': None, 'suppress_warnings': True, 'error_action': 'trace', 'trace': False, 'random': False, 'random_state': None, 'n_fits': 10, 'return_valid_fits': False, 'out_of_sample_size': 0, 'scoring': 'mse', 'scoring_args': None, 'with_intercept': 'auto', 'sarimax_kwargs': None, 'fit_args': None}

#### Processing Config file
Dependent variable, date variable, modeling granularity & other related modeling details are provided in the form of a config file. Each TS Algorithm, and the related hyperparameter values to be tried, should be given in the config file.

In [0]:
%run ../../../0_Config.ipynb

In [0]:
logger.info("Config file read")
assert set(app_config["Algorithms"]["SARIMAX"]["Hyperparameters"]['__init__'].keys()).\
           issubset(set(default_hpps_init.keys())),\
           'keys supplied by the user for the SARIMAX Algorithm under __init__ method must be valid'
assert set(app_config["Algorithms"]["SARIMAX"]["Hyperparameters"]['fit'].keys()).\
           issubset(set(default_hpps_fit.keys())),\
           'keys supplied by the user for the SARIMAX Algorithm under fit method must be valid'

# For exporting the config file
temp_config = app_config.copy()

In [0]:
def frange(start,stop,step= 1):
    l = []
    i = start
    while(i < stop):
        l.append(round(i,len(str(step))))
        i = i+step
    return l

def drange(hyperparameters):
    l=[]
    for key in hyperparameters.keys():
        val = hyperparameters[key]
        if 'range' in val:
            val = val.replace('range','frange')
            new_str = 'total_list = '  + val
            _locals = locals()
            exec(new_str,globals(),_locals)
            without_dup = list(set(_locals['total_list']))
            hyperparameters[key] = without_dup
    return hyperparameters

In [0]:
fit_ = drange(app_config['Algorithms']['SARIMAX']['Hyperparameters']['fit'])
init_ = drange(app_config['Algorithms']['SARIMAX']['Hyperparameters']['__init__'])
for key in init_.keys():
    if(key in fit_.keys()):
        fit_[key] = list(set(fit_[key]+init_[key]))
    else:
        fit_[key] = list(init_[key])
if("endog" in fit_.keys()):
    del fit_["endog"]
if("exog" in fit_.keys()):
    del fit_["endog"]
    
fit_new = {}
for key in fit_.keys():
    temp = []
    for val in fit_[key]:
        if(type(val) == list):
            val = str(val)
        if((val!='None') and (val!='Null') and (val!=None)):
            temp.append(val)
    if(len(temp)>0):
        fit_new[key] = temp
    
init_arima = app_config["Algorithms"]["SARIMAX"]['Auto_ARIMA']
init_arima_new = {}
for key in init_arima.keys():
    val = init_arima[key]
    if((val!='None') and (val!='Null') and (val!=None)):
        init_arima_new[key] = val
        
if app_config["Algorithms"]["SARIMAX"]['Auto_ARIMA']['use_auto_arima'] == True:
    if "order" in fit_new:
        print("While using Auto ARIMA feature, order should not be mentioned in the init section of Hyperparameters, will not be considering the order from the init section")
    fit_new["order"] = ['[0,0,0]'] #creating a dummy value, will be using only the best p,d,q values from auto arima
    if('seasonal' in init_arima_new.keys()):
        if(init_arima_new['seasonal']==True):
            fit_new["order"] = ['[1,1,1,12]'] #creating a dummy value, will be using only the best P,D,Q,m values from auto arima
    
app_config['Algorithms']['SARIMAX']['Hyperparameters'] = fit_new

While using Auto ARIMA feature, order should not be mentioned in the init section of Hyperparameters, will not be considering the order from the init section

In [0]:
# Create the algo directory for storing the results
output_directory = app_config['output_dir_path']
root_dir = "Modeling_Results"
algorithm = "SARIMAX"
algo_path = os.path.join(output_directory,root_dir,algorithm)
if not os.path.exists(algo_path):
    os.makedirs(algo_path)
logger.info("Created algorithm directory")    

logs_path = os.path.join(output_directory,root_dir,'logs',algorithm)
if not os.path.exists(logs_path):
    os.makedirs(logs_path)
logger.info("Created logs directory")

config_path = os.path.join(app_config['output_dir_path'],"Modeling_Results","config")
if not os.path.exists(config_path):
    os.makedirs(config_path)
logger.info("Created config directory")

#### Broadcasting the required variables
Variables suffixed with "_conf" are taken from the config file

In [0]:
hyperparameters_conf = dict(app_config["Algorithms"]["SARIMAX"]["Hyperparameters"])
# print(hyperparameters_conf)

modeling_granularity_conf = app_config["modeling_granularity"]
# print(modeling_granularity_conf)

# Rename Start date and DV config
dv_config = app_config["dependent_variable"]
ds_config = app_config["date_var"]

# pos and neg corr broadcast
corr_config = dict(app_config['Algorithms']['SARIMAX']['exogenous_variables'])
corr_config_broadcast = dotsi.Dict({"value":corr_config})

# Eval metric broadcast
broadcast_metric = dotsi.Dict({"value":app_config["validation"]["metric"]})
broadcast_tracking = dotsi.Dict({"value":app_config['tracking']})
broadcast_test_periods = dotsi.Dict({"value":app_config["validation"]["no_of_backtesting_test_periods"]})

broadcast_granularity = dotsi.Dict({"value":modeling_granularity_conf})
broadcast_hyper_parameters = dotsi.Dict({"value":hyperparameters_conf})

broadcast_autoarima_parameters = dotsi.Dict({"value":init_arima_new})

logger.info("Broadcasted the required variables")

In [0]:
# Reading feature selected output and using the significant variables as idvs in modeling
feature_selection_info = app_config['Algorithms']['SARIMAX']['feature_selection']
broadcast_use_features = dotsi.Dict({"value":feature_selection_info['use_feature_selected_idvs']})
if(feature_selection_info['use_feature_selected_idvs']):
    if(feature_selection_info['approach']=='lasso_cvglmnet'):
        output_folder = app_config['output_dir_path']+"/Feature_Selection/Lasso/"
    # Reading the latest input file based on timestamp
    coeff_op_files = [file for file in os.listdir(output_folder)]
    coeff_op_files = [file.replace(".csv","") for file in coeff_op_files]
    version_dates = [datetime.strptime(x.split('(')[1].replace(')',''), '%Y-%m-%d-%H-%M-%S') for x in coeff_op_files]
    max_date = max(version_dates)
    max_date = max_date.strftime('%Y-%m-%d-%H-%M-%S')
    req_file_name = [x for x in coeff_op_files if max_date in x]
    coeff_op_file_path = os.path.join(output_folder,req_file_name[0]+".csv")
    print(coeff_op_file_path)

    # Reading the data
    coeff_df = pd.read_csv(coeff_op_file_path)
    coeff_df = coeff_df[coeff_df['status']=='success']
    # print(coeff_df.shape)
    coeff_df[modeling_granularity_conf] = coeff_df[modeling_granularity_conf].astype(str)
    idvs_len = len(feature_selection_info['must_have_idvs'])
    if(idvs_len>0):
        temp1 = coeff_df[modeling_granularity_conf].drop_duplicates()
        temp1['temp'] = 1
        temp2 = pd.DataFrame({'IDV':feature_selection_info['must_have_idvs']})
        temp2['temp'] = 1
        temp = temp1.join(temp2, on = 'temp', how ='left')
        req_cols = modeling_granularity_conf + ['IDV']
        coeff_df = coeff_df.drop_duplicates()
    coeffs_broadcast = dotsi.Dict({"value":coeff_df})
# display(coeff_df)

In [0]:
# UDF of Auto-ARIMA
def get_pdq_values(df_data: pd.DataFrame) -> pd.DataFrame:
    """Function to extract the best p,d,q values for the given input data using the broadcasted information from the config file

    Parameters
    ----------
    df_data : pd.DataFrame
        The dataset containing values for all the required variables

    Returns
    -------
    pd.DataFrame
        Returns a dataframe with the granularity & p,d,q values
    """
    
    try:
        
        results_pd = {}
        df_data = df_data.sort_values(by=['ds'],ascending=True)
        if(broadcast_use_features.value==True):
            # Reading regressors from feature selection
            coeffs_df = coeffs_broadcast.value
            for x in broadcast_gran:
                coeffs_df = coeffs_df[coeffs_df[x] == df_data[x].iloc[0]]
            regressors = list(coeffs_df['IDV'].values)
        else:
            corr_var = corr_config_broadcast.value
            regressors = list(set(corr_var["positive_corr"] + corr_var["negative_corr"]+corr_var['uncertain_corr']))
        
        # broadcast_granularity
        broadcast_gran = broadcast_granularity.value
        
        # Updating the default arguments with the parameters provided in the config
        hp_config = broadcast_autoarima_parameters.value
        def_args_arima = get_default_args(pm.auto_arima)
        
        for x in hp_config:
            if(x in def_args_arima.keys()):
                def_args_arima[x] = hp_config[x]
        
        def_args_arima['y'] = df_data[['y']]
        
        if hp_config['consider_exogenous_variables'] == True:
            def_args_arima['X'] = df_data[regressors]
            
        model = pm.auto_arima(**def_args_arima)
        
        for x in broadcast_gran:
            results_pd[x] = df_data[x].astype(str).iloc[0]
            
        results_pd['order'] = str(list(model.order))
        if hp_config['seasonal'] == True:
            results_pd['seasonal_order'] = str(list(model.seasonal_order))
        else:
            results_pd['seasonal_order'] = ''
            
        results_pd['status'] = 'success'
        return pd.DataFrame.from_dict([results_pd])
    
    except Exception as e:
        results_pd = {}
        
        for x in broadcast_gran:
            results_pd[x] = df_data[x].astype(str).iloc[0]

        results_pd['order'] = ''
        results_pd['seasonal_order'] = ''
        results_pd['status'] = str(e)
        return pd.DataFrame.from_dict([results_pd])

#### Pandas UDF for Backtesting
The UDF gets executed in multiple worker nodes to parallelize the backtesting process. All the broadcasted variables are accessed within the UDF as and when required

In [0]:
def get_prediction_UDF(df_data: pd.DataFrame) -> pd.DataFrame:
    """Function to perform backtesting for the given input data using the broadcasted information from the config file

    Parameters
    ----------
    df_data : pd.DataFrame
        The dataset containing values for all the required variables

    Returns
    -------
    pd.DataFrame
        Returns a dataframe with the granularity,date,hyperparameters,window and performance metrics
    """
    try:
        train_index_start = df_data["train_index_start"].iloc[0]
        train_index_end = df_data["train_index_end"].iloc[0]
        test_i = df_data["test_index_end"].iloc[0]
        window_no = df_data["window_no"].iloc[0]
        
        df_data = df_data.sort_values(by=['ds'],ascending=True)
        df_data = df_data.iloc[train_index_start:test_i].reset_index(drop=True)

        # broadcast_granularity
        broadcast_gran = broadcast_granularity.value
        
        broadcast_autoarima = broadcast_autoarima_parameters.value

        # Train - test split
        test_periods = int(broadcast_test_periods.value)
        train = df_data.iloc[:-test_periods]
        test = df_data.iloc[-test_periods:]

        # Updating the default arguments with the parameters provided in the config
        hp_config = broadcast_hyper_parameters.value
        def_args_init = get_default_args(SARIMAX.__init__)
        
        for x in hp_config:
            if(x in def_args_init.keys()):
                temp_val = df_data[x].iloc[0]
                if(type(temp_val)==str):
                    if('[' in temp_val):
                        temp_val = eval(temp_val)
                def_args_init[x] = temp_val
        def_args = get_default_args(SARIMAX.fit)
        for x in hp_config:
            if(x in def_args.keys()):
                temp_val = df_data[x].iloc[0]
                if(type(temp_val)==str):
                    if('[' in temp_val):
                        temp_val = eval(temp_val)
                def_args[x] = temp_val
          
        if(broadcast_use_features.value==True):
            # Reading regressors from feature selection
            coeffs_df = coeffs_broadcast.value
            for x in broadcast_gran:
                coeffs_df = coeffs_df[coeffs_df[x] == df_data[x].iloc[0]]
            regressors = list(coeffs_df['IDV'].values)
        else:
            corr_var = corr_config_broadcast.value
            regressors = list(set(corr_var["positive_corr"] + corr_var["negative_corr"]+corr_var['uncertain_corr']))

            temp_list1 = []
            # Removing regressors based on the correlation
            if(corr_var["consider_correlation"]):   
                for x in corr_var["positive_corr"]:
                    if(train[['y',x]].corr().iloc[0][1]<0):
                        temp_list1.append(x)
                for x in corr_var["negative_corr"]:
                    if (x not in temp_list1):
                        if(train[['y',x]].corr().iloc[0][1]>0):
                            temp_list1.append(x)   
                regressors = list(set(regressors) - set(temp_list1))

            # Checking for variance in the regressor
            temp_list = []
            if len(regressors)>0:
                for ex_var in regressors:
                    mean = train[ex_var].mean()
                    std = train[ex_var].std()
                    if mean == 0:
                        if std <= 0.001:
                            temp_list.append(ex_var)
                    else:
                        if std/mean <= 0.01:
                            temp_list.append(ex_var)
            regressors = list(set(regressors) - set(temp_list))

        def_args_init['endog'] = train['y']
        if(len(regressors)>0):
            def_args_init['exog'] = train[regressors]
            # Calling the SARIMAX constructor with the hyperparameters of interest  
            m = SARIMAX(**def_args_init)
            m = m.fit(**def_args)
            test['yhat'] = m.predict(start = test.index.min(),end = test.index.max(),exog=test[regressors])
        else:
            # Calling the SARIMAX constructor with the hyperparameters of interest  
            m = SARIMAX(**def_args_init)
            m = m.fit(**def_args)
            test['yhat'] = m.predict(start = test.index.min(),end = test.index.max())
           
        results_pd = test[broadcast_gran+['ds', 'y', 'yhat']].reset_index(drop = True)
        y_pred = results_pd['yhat']
        y_true = results_pd['y']

        # correct y_pred to 0 if -ve
        y_pred = np.where(y_pred < 0, 0, y_pred)

        # to handle erroneous results epsilon is set to 1.
        epsilon = 1

        results_pd['mape'] = np.mean(np.abs(y_true - y_pred) / np.maximum(np.abs(y_true), epsilon))*100
        results_pd['wmape'] = np.sum(np.abs(y_true - y_pred)) / np.maximum(np.sum(np.abs(y_true)),epsilon)*100
        results_pd['bias'] = np.mean((y_true - y_pred))
        results_pd['tracking_signal'] = np.sum((y_true - y_pred)) / np.mean(np.abs(y_true - y_pred))
        results_pd['mae'] = mean_absolute_error(y_true, y_pred)
        results_pd['rmse']=np.sqrt(mean_squared_error(y_true, y_pred))

        # to adhere to returned Schema
        for x in broadcast_gran:
            results_pd[x] = results_pd[x].astype(str)

        # append Hyper Parameter information
        for x in hp_config:
            results_pd[x] = df_data[x].iloc[0]
            
        results_pd["window"] = str(str(train_index_start)+" "+str(train_index_end)+" "+str(test_i)+" "+str(window_no))

        # Sales or Quantity can't be negative hence
        results_pd["yhat"] = np.where(results_pd["yhat"]<0,0,results_pd["yhat"])

        results_pd['status'] = 'success'
        return results_pd
    
    except Exception as e:
        results_pd = pd.DataFrame(columns = [['ds', 'y', 'yhat','mape','wmape','bias','tracking_signal','mae','rmse','window']+\
                        list(broadcast_hyper_parameters.value.keys()) + ['status'] + broadcast_granularity.value],index = range(1))
        results_pd[broadcast_granularity.value] = df_data[broadcast_granularity.value].head(1).reset_index(drop = True)
        for x in broadcast_granularity.value:
            results_pd[x] = results_pd[x].astype(str)
        results_pd['status'] = str(e)
        return results_pd

#### Hyperparameter grid
The following function creates a cross product of all the hyperparameters provided in the config file and returns a hyperparamter grid

In [0]:
# Creating a hyperparameter grid using the set of provided hyperparameters in the config file
def create_hyperparam_space(hp_space: dict) -> pd.DataFrame:
    """Function to create a hyperparameter grid using the set of provided hyperparameters in the config file to be used for Hyperparameter tuning

    Parameters
    ----------
    hp_space : dict
        The set of provided hyperparameters in the config file

    Returns
    -------
    pd.DataFrame
        Returns a hyperparameter grid created from the set of provided hyperparameters
    """    
    df_list = list()
    for x in hp_space:
      df_list.append(pd.DataFrame({x:hp_space[x]}))

    space=df_list[0]

    for x in df_list[1:]:
        space = space.merge(x,how="cross")

    return space

#### Backtesting parallelized using UDF
Based on the Backtesting algorithm (sliding_window/expanding_window), training percentage & test periods specified in the config file, Hyperparameter tuning is performed

In [0]:
def hyperparameter_tuning(
    algorithm: str,
    train_data: pd.DataFrame,
    hyperparam_space: dict,
    modeling_granularity_conf: list,
    train_percentage: float,
    backtesting_test_periods: int,
    test_periods: int,
    stride: int,
    modelling_func: pd.DataFrame = get_prediction_UDF,
    pdq_results: pd.DataFrame = None,
) -> pd.DataFrame:
    
    """Function to perform the hyperparameter tuning

    Parameters
    ----------
    algorithm : str
        the algorithm with which the hyperparameter tuning is to be performed(sliding_window/expanding_window)
    train_data : pd.DataFrame
        the modelling dataset
    hyperparam_space : dict
        hyperparameter grid which was created using the set of hyperparameter from config file
    modeling_granularity_conf : list
        The list of the granularity variables at which the models will be built
    train_percentage : float
        The percentage of data points for the training
    backtesting_test_periods : int
        number of time periods to score in each iteration of backtesting
    test_periods : int
        number of time periods to score in final model building
    stride : int
        number of time periods to stride in each iteration of Backtesting
    modelling_func : pd.DataFrame, optional
        the function name to perform backtesting for the given dataset, by default get_prediction_UDF
    pdq_results : pd.DataFrame, optional
        the pdq results from auto arima
    Returns
    -------
    pd.DataFrame
        Returns the dataframe containing the hyperparameter set of modelling granularity x hyperparameter set x window level
    """
    if algorithm == "expanding_window":
        gbcp = list(modeling_granularity_conf) + list(hyperparam_space.columns)
        unique_skuXds = train_data[modeling_granularity_conf+["#total_weeks"]].drop_duplicates().reset_index(drop = True)
        
        final_list = []
        gran_len = len(modeling_granularity_conf)
        for row1 in range(0,len(unique_skuXds)): 
            Total_weeks = unique_skuXds.loc[row1,'#total_weeks']
            train_interval = int((Total_weeks-test_periods) * train_percentage)
            j = 0
            train_period_ends = Total_weeks-test_periods
            for train_i in range(train_interval,train_period_ends,stride):
                if(train_i+backtesting_test_periods <=train_period_ends):
                    test_i = train_i+backtesting_test_periods
                    final_list.append([unique_skuXds.iloc[row1,index] for index in range(gran_len)] + [0,train_i,train_i+backtesting_test_periods,j+1])
                    j += 1
                    
        # create all windows combination.
        df_windows = pd.DataFrame([tuple(x) for x in final_list],columns =modeling_granularity_conf+\
                                  ['train_index_start','train_index_end','test_index_end','window_no'])
        f_df = train_data.merge(df_windows,on=modeling_granularity_conf,how="left")
        f_df['temppp'] = 1
        hyperparam_space['temppp'] = 1
        f_df = f_df.merge(hyperparam_space,on='temppp',how="left")
        
        if pdq_results is not None:
            if('order' in f_df.columns):
                del(f_df['order'])
            params_autoarima = broadcast_autoarima_parameters.value
            if('seasonal' in params_autoarima.keys()):
                if((params_autoarima['seasonal']==True) and ('seasonal_order' in f_df.columns)):
                    del(f_df['seasonal_order'])
                else:
                    del(pdq_results['seasonal_order'])
            f_df = f_df.merge(pdq_results[[column for column in pdq_results.columns if column not in ['success']]], \
                              on = modeling_granularity_conf, how = 'left')
        
        f_df['gran_tempp'] = f_df[gbcp+["train_index_start","train_index_end","test_index_end","window_no"]].astype(str).sum(axis=1)
        unique_pdts = f_df['gran_tempp'].unique()
        new_results = pd.DataFrame()
        for pdt in unique_pdts:
            new_results = pd.concat([new_results,modelling_func(f_df[f_df['gran_tempp']==pdt])])
        return new_results
    
    elif algorithm == "sliding_window":
        gbcp = list(modeling_granularity_conf) + list(hyperparam_space.columns)
        unique_skuXds = train_data[modeling_granularity_conf+["#total_weeks"]].drop_duplicates().reset_index(drop = True)
        
        final_list = []
        gran_len = len(modeling_granularity_conf)
        for row1 in range(0,len(unique_skuXds)): 
            Total_weeks = unique_skuXds.loc[row1,'#total_weeks']
            train_interval = int((Total_weeks-test_periods) * train_percentage)
            j = 0
            train_period_ends = Total_weeks-test_periods
            train_index_start = 0
            for train_i in range(train_interval,train_period_ends,stride):
                if(train_i+backtesting_test_periods <=train_period_ends):
                    test_i = train_i+backtesting_test_periods
                    final_list.append([unique_skuXds.iloc[row1,index] for index in range(gran_len)] + \
                                      [train_index_start,train_i,train_i+backtesting_test_periods,j+1])
                    j += 1
                    train_index_start = train_index_start+stride
                    
        # create all windows combination.
        df_windows = pd.DataFrame([tuple(x) for x in final_list],\
                                  columns =modeling_granularity_conf+['train_index_start','train_index_end','test_index_end','window_no'])
        f_df = train_data.merge(df_windows,on=modeling_granularity_conf,how="left")
        f_df['temppp'] = 1
        hyperparam_space['temppp'] = 1
        f_df = f_df.merge(hyperparam_space,on='temppp',how="left")
        
        if pdq_results is not None:
            if('order' in f_df.columns):
                del(f_df['order'])
            params_autoarima = broadcast_autoarima_parameters.value
            if('seasonal' in params_autoarima.keys()):
                if((params_autoarima['seasonal']==True) and ('seasonal_order' in f_df.columns)):
                    del(f_df['seasonal_order'])
                else:
                    del(pdq_results['seasonal_order'])
                
            f_df = f_df.merge(pdq_results[[column for column in pdq_results.columns if column not in ['success']]], \
                              on = modeling_granularity_conf, how = 'left')
            
        f_df['gran_tempp'] = f_df[gbcp+["train_index_start","train_index_end","test_index_end","window_no"]].astype(str).sum(axis=1)
        unique_pdts = f_df['gran_tempp'].unique()
        new_results = pd.DataFrame()
        for pdt in unique_pdts:
            new_results = pd.concat([new_results,modelling_func(f_df[f_df['gran_tempp']==pdt])])
        return new_results

#### Loading the latest Missing_value_treatment file
##### Please update the reading path with the required data path if "Missing value treatment" was not run

In [0]:
# Reading the latest input file based on timestamp
all_files = [file for file in os.listdir(app_config['output_dir_path']+"/Data_Processing/Missing_value_treatment")]
missing_op_files = [file for file in all_files if "Missing_value_treatment_results (" in file]
missing_op_files = [file.replace(".csv","") for file in missing_op_files]
version_dates = [datetime.strptime(x.split('(')[1].replace(')',''), '%Y-%m-%d-%H-%M-%S') for x in missing_op_files]
max_date = max(version_dates)
max_date = max_date.strftime('%Y-%m-%d-%H-%M-%S')
req_file_name = [x for x in missing_op_files if max_date in x]
missing_op_file_path = os.path.join(app_config['output_dir_path']+"/Data_Processing/Missing_value_treatment",req_file_name[0] + ".csv")
# print(missing_op_file_path)

# Reading the data
df = pd.read_csv(missing_op_file_path)
# print(df.shape)

df.rename(columns = {ds_config:"ds", dv_config:"y"}, inplace = True)
logger.info("Data loaded")

df['ds'] = pd.to_datetime(df['ds'])
df[modeling_granularity_conf] = df[modeling_granularity_conf].astype(str)
    
# Getting the total number of weeks for each time series
temp_df = df.groupby(modeling_granularity_conf).agg({'ds':'count'}).rename(columns={'ds': '#total_weeks'}).reset_index()
df = df.merge(temp_df, on = modeling_granularity_conf ,how = "left")

# 2. Create the hyperparameter space
hpspace = create_hyperparam_space(hyperparameters_conf)
logger.info("Created hyperparameter grid")

# 3. Running Auto Arima if use_auto_arima is yes
if app_config["Algorithms"]["SARIMAX"]['Auto_ARIMA']['use_auto_arima'] == True:
    df['gran_tempp'] = df[modeling_granularity_conf].astype(str).sum(axis=1)
    unique_pdts = df['gran_tempp'].unique()
    best_pdq_results = pd.DataFrame()
    for pdt in unique_pdts:
        best_pdq_results = pd.concat([best_pdq_results,get_pdq_values(df[df['gran_tempp']==pdt])])
        
    best_pdq_results.to_csv(algo_path+"/Best_pdq_values_from_auto_arima ("+datetime.today().strftime('%Y-%m-%d-%H-%M-%S')+").csv", index = False)
    
    # Reading the latest input file based on timestamp
    all_files = [file for file in os.listdir(algo_path)]
    pdq_op_files = [file for file in all_files if "Best_pdq_values_from_auto_arima (" in file]
    pdq_op_files = [file.replace(".csv","") for file in pdq_op_files]
    version_dates = [datetime.strptime(x.split('(')[1].replace(')',''), '%Y-%m-%d-%H-%M-%S') for x in pdq_op_files]
    max_date = max(version_dates)
    max_date = max_date.strftime('%Y-%m-%d-%H-%M-%S')
    req_file_name = [x for x in pdq_op_files if max_date in x]
    pdq_file_path = os.path.join(algo_path,req_file_name[0] + ".csv")
    # print(pdq_file_path)

    # Reading the data
    best_pdq_results = pd.read_csv(pdq_file_path)
    best_pdq_results[modeling_granularity_conf] = best_pdq_results[modeling_granularity_conf].astype(str)
    print(best_pdq_results)

# 4. HP tuning based on algorithm of choice
if app_config["Algorithms"]["SARIMAX"]['Auto_ARIMA']['use_auto_arima'] == True:
    df_f = hyperparameter_tuning(app_config['validation']['backtesting']['algorithm'],\
                                 df,hpspace,modeling_granularity_conf,\
                                 app_config['validation']['train_percentage'],\
                                 app_config['validation']['no_of_backtesting_test_periods'],\
                                 app_config['validation']['no_of_test_periods'],\
                                 app_config['validation']['backtesting']['stride'],modelling_func = get_prediction_UDF, pdq_results = best_pdq_results)
else:
    df_f = hyperparameter_tuning(app_config['validation']['backtesting']['algorithm'],\
                                 df,hpspace,modeling_granularity_conf,\
                                 app_config['validation']['train_percentage'],\
                                 app_config['validation']['no_of_backtesting_test_periods'],\
                                 app_config['validation']['no_of_test_periods'],\
                                 app_config['validation']['backtesting']['stride'],modelling_func = get_prediction_UDF)
df_f.to_csv(algo_path+"/Backtesting_results_window_level ("+datetime.today().strftime('%Y-%m-%d-%H-%M-%S')+").csv", index = False)
logger.info("Completed Backtesting")

/databricks/python/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: divide by zero encountered in reciprocal
 return np.roots(self.polynomial_reduced_ar)**-1
/databricks/python/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: divide by zero encountered in reciprocal
 return np.roots(self.polynomial_reduced_ma)**-1
/databricks/python/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: divide by zero encountered in reciprocal
 return np.roots(self.polynomial_reduced_ar)**-1
/databricks/python/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: divide by zero encountered in reciprocal
 return np.roots(self.polynomial_reduced_ma)**-1
/databricks/python/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: divide by zero encountered in reciprocal
 return np.roots(self.polynomial_reduced_ar)**-1
/databricks/python/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: divide by zero encountered in reciprocal
 return np.roots(self.polynomial_reduced_ma)**-1
/databricks/python/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: divide by zero encountered in reciprocal
 return np.roots(self.polynomial_reduced_ar)**-1
/databricks/python/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: divide by zero encountered in reciprocal
 return np.roots(self.polynomial_reduced_ma)**-1
/databricks/python/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: divide by zero encountered in reciprocal
 return np.roots(self.polynomial_reduced_ar)**-1
/databricks/python/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: divide by zero encountered in reciprocal
 return np.roots(self.polynomial_reduced_ma)**-1
 Div_No Store_No Base_UPC order seasonal_order status
0 24 8 2200015934 [0, 0, 0] [0, 0, 0, 0] success
1 24 14 4000042206 [1, 0, 0] [0, 0, 0, 0] success
2 24 15 4000046410 [1, 0, 0] [0, 0, 0, 0] success
3 24 17 4000000032 [1, 0, 0] [0, 0, 0, 0] success
4 24 53 4000000263 [1, 0, 3] [0, 0, 0, 0] success
5 24 55 4000000051 [0, 0, 0] [0, 0, 0, 0] success
6 24 55 4000005851 [0, 0, 0] [0, 0, 0, 0] success
/databricks/python/lib/python3.8/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method lbfgs is: m, pgtol, factr, maxfun, epsilon, approx_grad, bounds, loglike_and_score. The list of unsupported keyword arguments passed include: kwargs. After release 0.14, this will raise.
 warnings.warn(
<command-3460216454771168>:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 test['yhat'] = m.predict(start = test.index.min(),end = test.index.max(),exog=test[regressors])
/databricks/python/lib/python3.8/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method lbfgs is: m, pgtol, factr, maxfun, epsilon, approx_grad, bounds, loglike_and_score. The list of unsupported keyword arguments passed include: kwargs. After release 0.14, this will raise.
 warnings.warn(
<command-3460216454771168>:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 test['yhat'] = m.predict(start = test.index.min()

### Choosing the best hyperparameters

In [0]:
# Reading the latest file based on timestamp
all_files = [file for file in os.listdir(algo_path)]
backtesting_files = [file for file in all_files if "Backtesting_results_window_level (" in file]
backtesting_files = [file.replace(".csv","") for file in backtesting_files]
version_dates = [datetime.strptime(x.split('(')[1].replace(')',''), '%Y-%m-%d-%H-%M-%S') for x in backtesting_files]
max_date = max(version_dates)
max_date = max_date.strftime('%Y-%m-%d-%H-%M-%S')
req_file_name = [x for x in backtesting_files if max_date in x]
backtesting_results_file_path = os.path.join(algo_path,req_file_name[0] + ".csv")
# print(backtesting_results_file_path)

In [0]:
# Reading the results of backtesting
df = pd.read_csv(backtesting_results_file_path)
df = df[df['status'] == 'success']
# display(df)

In [0]:
# Roll up the metrics at Modeling granularity x Hyperparameter space x window level
window_level = modeling_granularity_conf + list(hyperparameters_conf.keys()) + ["window"]
# print(window_level)
window_level_results = df.groupby(window_level)[["mape","wmape","bias","tracking_signal","mae","rmse"]].min().reset_index()

In [0]:
# Roll up the metrics at Modeling granularity x Hyperparameter space level
hyperparam_level = modeling_granularity_conf + list(hyperparameters_conf.keys())
hyperparam_level_results = window_level_results.groupby(hyperparam_level)[["mape","wmape","bias","tracking_signal","mae","rmse"]].mean().reset_index()

In [0]:
hyperparam_level_results.to_csv(algo_path + "/Backtesting_results_hyperparameter_level (" + datetime.today().strftime('%Y-%m-%d-%H-%M-%S')+").csv", index = False)
logger.info("Exported backtesting results")

#### Pandas UDF for getting best hyperparameters & MLFlow tracking

In [0]:
hyperparam_level_results['algorithm'] = 'SARIMAX'
hyperparam_level_results['result_type'] = 'backtesting'

In [0]:
def get_best_hyperparameters(final_hyperparam_df: pd.DataFrame) -> pd.DataFrame:
    """Function for getting the best hyperparameters results for each modeling granularity along with MLFlow tracking using the broadcasted information from the config file

    Parameters
    ----------
    final_hyperparam_df : pd.DataFrame
        Dataframe containing the results of Backtesting 

    Returns
    -------
    pd.DataFrame
        Dataset with best hyperparameter set for each modeling granularity
    """
    try:
        results_pd = {}

        # Get the modeling granularity
        broadcast_gran = broadcast_granularity.value
        # Get the experiment id
        tracking_value = broadcast_tracking.value.copy()

        # To adhere to defined schema
        for x in broadcast_gran:
            results_pd[x] = final_hyperparam_df[x].astype(str).iloc[0]

        hp_config = broadcast_hyper_parameters.value
        granularity = broadcast_granularity.value
        
        if tracking_value["tracking_needed"] == True:
            if tracking_value["type"] != "Managed":
                if tracking_value["tracking_uri"] is not None:
                    mlflow.set_tracking_uri("file:" + tracking_value["tracking_uri"])
                    experiment_id = mlflow.set_experiment(tracking_value["mlflow_experiment_id"])
                    tracking_value['mlflow_experiment_id'] = experiment_id.experiment_id

            # Add MLFlow code here
            with mlflow.start_run(experiment_id=tracking_value["mlflow_experiment_id"]):
                for x in ["algorithm",'result_type'] + broadcast_gran:
                    mlflow.log_param(x, final_hyperparam_df[x].iloc[0])
                for row in range(0, len(final_hyperparam_df)):
                    with mlflow.start_run(experiment_id=tracking_value["mlflow_experiment_id"], nested=True):
                        for x in hp_config:
                            mlflow.log_param(x, final_hyperparam_df[x].iloc[row])
                        for x in ["mape","wmape","bias","tracking_signal","mae","rmse"]:
                            mlflow.log_metric(x, final_hyperparam_df[x].iloc[row])

        metric = broadcast_metric.value

        # Sort using the metric of interest
        if metric in ["wmape", "mape", "mad", "mae", "rmse"]:
            final_hyperparam_df = final_hyperparam_df.sort_values(
                metric, ascending=True
            )
        elif metric in ["tracking_signal","bias"]:
            final_hyperparam_df.sort_values(metric, ascending=True,key=abs, inplace=True)
        else:
            final_hyperparam_df.sort_values(metric, ascending=True, inplace=True)

        # Adding the best hyperparameter and related metrics
        for x in hp_config:
            results_pd[x] = final_hyperparam_df[x].iloc[0]

        for x in ["mape", "wmape", "bias", "tracking_signal", "mae", "rmse"]:
            results_pd[x] = final_hyperparam_df[x].iloc[0]

        results_pd["status"] = "success"
        return pd.DataFrame.from_dict([results_pd])

    except Exception as e:
        results_pd = pd.DataFrame(
            columns=[["mape", "wmape", "bias", "tracking_signal", "mae", "rmse"]+ list(broadcast_hyper_parameters.value.keys())+ ["status"]+ broadcast_granularity.value],\
            index=range(1))
        results_pd[broadcast_granularity.value] = final_hyperparam_df[broadcast_granularity.value].head(1).reset_index(drop=True)
        for x in broadcast_granularity.value:
            results_pd[x] = results_pd[x].astype(str)
        results_pd["status"] = str(e)
        return results_pd

In [0]:
hyperparam_level_results['gran_tempp'] = hyperparam_level_results[modeling_granularity_conf].astype(str).sum(axis=1)
unique_pdts = hyperparam_level_results['gran_tempp'].unique()
best_hyperparam_results = pd.DataFrame()
for pdt in unique_pdts:
    best_hyperparam_results = pd.concat([best_hyperparam_results,get_best_hyperparameters(hyperparam_level_results[hyperparam_level_results['gran_tempp']==pdt])])

#### Writing the best hyperparameter results

In [0]:
best_hyperparam_results.to_csv(algo_path + "/Best_hyperparameters (" + datetime.today().strftime('%Y-%m-%d-%H-%M-%S')+").csv", index = False)
logger.info("Exported best hyperparameter results")

In [0]:
# Exporting config file
config_file_name = "config_for_exp_id_"+str(broadcast_tracking.value['mlflow_experiment_id']) + " (" +datetime.today().strftime('%Y-%m-%d-%H-%M-%S-%f')[:-3]+").yml"
config_path1 = os.path.join(config_path,config_file_name)
with open(config_path1, 'w') as file:
    yaml.dump(temp_config, file, default_flow_style=False,sort_keys=False)

In [0]:
# Move from tmp directory to req. location in datalake
import platform
plat_sys = platform.system()

if(plat_sys!='Windows'):
    log_file = log_file.replace(' (', '\ \(').replace(')','\)')
    os.system('mv /tmp/{0} {1}'.format(log_file,logs_path))